In [1]:
cd Advent_of_code/

[Errno 2] No such file or directory: 'Advent_of_code/'
/mnt/c/Users/rasmu/Dropbox/Uni/AdventOfCode


# December 1.

In [2]:
sizes = [sum([int(x) for x in section.split("\n")]) 
         for section in open("day1.txt").read().split("\n\n")]
print("svar 1:", max(sizes))
print("svar 2:", sum(sorted(sizes)[-3:]))

svar 1: 71502
svar 2: 208191


# December 2.

In [3]:
values = {"A": 1, "B": 2, "C": 3, "X" : 1, "Y": 2, "Z" : 3}
print("svar 1:", sum([[3, 6, 0][values[b] - values[a]] + values[b] 
     for a, _, b, _ in open("day2.txt").readlines()]))
print("svar 2:", sum([("XYZ".index(b))*3+("ABC".index(a)+"XYZ".index(b)-1)%3+1
        for a, _, b, _ in open("day2.txt").readlines()]))

svar 1: 15691
svar 2: 12989


# December 3.

In [4]:
overlap = lambda x: (set(x[:len(x)//2]) & set(x[len(x)//2:]))
priority = lambda x: ord(x)-(38 if x.isupper() else 96)
print("svar 2:", sum([priority(*overlap(x)) for x in open("day3.txt").read().split("\n")]))
types = [set(x) for x in open("day3.txt").read().split("\n")]
print("svar 2:", sum([priority(*(types[3*i+0] & types[3*i+1] & types[3*i+2])) for i in range(len(types)//3)]))

svar 2: 7872
svar 2: 2497


# December 4.

In [5]:
parse_range = lambda a,b: set(range(int(a), int(b)+1))
to_maps = lambda pair: [parse_range(*x.split("-")) for x in pair.split(",")]
fully_overlaps = lambda a,b: min(len(a),len(b)) == len(a & b)
partial_overlap = lambda a,b: len(a & b) > 0
pairs_as_maps = [to_maps(pair) for pair in open("day4.txt").read().split("\n")]
print("svar 1:", sum([fully_overlaps(a,b) for a, b in pairs_as_maps]))
print("svar 2:", sum([partial_overlap(a,b) for a, b in pairs_as_maps]))

svar 1: 487
svar 2: 849


# December 5.

In [6]:
initial, moves = open("day5.txt").read().split("\n\n")
moves_list = [[int(y) for y in move.split(" ")[1::2]] for move in moves.split("\n")]

def parse_initial(initial):
    unfiltered_columns = list(zip(*initial.split("\n")[:-1]))[1::4]
    return [[x for x in row[::-1] if x != " "] for row in unfiltered_columns]
    
def move(stacks, a, b, count, orderer):
    stacks[b].extend(orderer(stacks[a][-count:]))
    stacks[a] = stacks[a][:-count]

for i in range(2):
    stacks = parse_initial(initial)
    for cnt, a,b in moves_list:
        move(stacks, a-1, b-1, cnt, lambda x: [x[::-1], x][i])
    print(f"svar {i+1}:", "".join([x[-1] for x in stacks]))

svar 1: HBTMTBSDC
svar 2: PQTJRSHWS


# December 6.

In [7]:
buffer = open("day6.txt").read()
for i, l in enumerate([4,14]):
    print(f"svar {i+1}:", min([i for i in range(l, len(buffer)) if len(set(buffer[i-l:i])) == l]))

svar 1: 1965
svar 2: 2773


# December 7.

In [8]:
file = open("day7.txt")
sizes = []
def sum_dict(file):
    size = 0
    while "$ cd ..\n" != (next_line := file.readline()):
        if not next_line:
            break
        if "$ cd" in next_line:
            size += sum_dict(file)
        elif next_line[:4] not in ["$ ls", "dir "]:
            size += int(next_line.split(" ")[0])
    sizes.append(size)
    return size
sum_dict(file)
print("svar 1:", sum([x for x in sizes if x <= 100000]))
print("svar 2:", min([x for x in sizes if x > max(sizes)-40000000]))

svar 1: 1306611
svar 2: 13210366


# December 8

In [9]:
import numpy as np
def visible_from_north(trees):
    return np.concatenate([np.ones((1,trees.shape[1])), trees[1:]>np.maximum.accumulate(trees)[:-1]])

def scenic_from_north(trees):
    return np.array([np.minimum(sum(np.minimum.accumulate(trees[i,1:-1] > trees[i+1:,1:-1]))+1, len(trees)-i-1) for i in range(1,len(trees)-1)])

trees = np.array([[*x] for x in open("day8.txt").read().split("\n")]).astype(int)
print("svar 1:", np.sum(sum([np.rot90(visible_from_north(np.rot90(trees, k)), -k) for k in (0,1,2,3)])>0))
print("svar 2:", np.max(np.prod([np.rot90(scenic_from_north(np.rot90(trees, k)), -k) for k in (0,1,2,3)],0)))

svar 1: 1787
svar 2: 440640


# December 9

In [10]:
def adjust_rope(positions, visited):
    for i in range(1,len(positions)):
        if np.hypot(*(positions[i]-positions[i-1])) >= 1.5:
            positions[i] += np.clip(positions[i-1] - positions[i], -1, 1)
    visited.add(tuple(positions[-1]))
            
directions = {"L":[-1, 0], "R":[1, 0], "U":[0, 1], "D":[0, -1]}
def follow_rope_instructions(positions):
    visited = {(0,0)}
    for line in open("day9.txt").readlines():
        orient, cnt = line.split()
        for i in range(int(cnt)):
            positions[0] += directions[orient]
            adjust_rope(positions, visited)
    return len(visited)

print("svar 1:", follow_rope_instructions(np.zeros((2,2),dtype=int)))
print("svar 2:", follow_rope_instructions(np.zeros((10,2),dtype=int)))

svar 1: 6376
svar 2: 2607


# December 10

In [11]:
x = [1]
for op in open("day10.txt").read().split("\n"):
    x.append(x[-1])
    if "noop" not in op:
        x.append(x[-1]+int(op.split()[1]))
x = np.array(x)
print("svar 1:", sum(x[19::40]*np.arange(20,240,40)))
display = lambda array: "\n".join(["".join(["#" if pos else "." for pos in row]) for row in array])
print("svar 2:\n", display((abs(np.arange(240)%40 - x[:-1]) < 2).reshape(6,40)))

svar 1: 12520
svar 2:
 ####.#..#.###..####.###....##..##..#....
#....#..#.#..#....#.#..#....#.#..#.#....
###..####.#..#...#..#..#....#.#....#....
#....#..#.###...#...###.....#.#.##.#....
#....#..#.#....#....#....#..#.#..#.#....
####.#..#.#....####.#.....##...###.####.
